# 💡 
If outlier exposure is used does a neural network perform better? The hypothesis is that it will learn a more general representation that won't be interfered with as much.

In [63]:
import typing
import torch
import torchvision.transforms as transforms 
import numpy as np
import torchvision.datasets

from random import shuffle

import torch.nn as nn
from torch import Tensor
from dataclasses import dataclass
from avalanche.benchmarks.classic.cmnist import SplitMNIST
from avalanche.training.templates.supervised import SupervisedTemplate
from avalanche.benchmarks.utils import AvalancheSubset, AvalancheDataset
from avalanche.benchmarks.scenarios.new_classes.nc_scenario import NCScenario
from avalanche.training.plugins.lwf import LwFPlugin
from avalanche.training.plugins.synaptic_intelligence import SynapticIntelligencePlugin



%load_ext autoreload
%autoreload 2

from conf import *
from experiment.experiment import Experiment, BaseHyperParameters
from functional.functional import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
class MySimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(28*28, 512),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.ReLU(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x)

In [88]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std



    # pass

dataset = torchvision.datasets.MNIST(
    DATASETS, 
    train=True, 
    download=True, 
    transform=transforms.Compose([
        transforms.ToTensor(),
    ]))

loader = torch.utils.data.DataLoader(dataset, batch_size=64)
mean, std = get_mean_and_std(loader)

print(f"mean {mean}, std {std}, n {len(dataset)}")

mean tensor([0.1307]), std tensor([0.3081]), n 60000


In [91]:
@dataclass
class HyperParams(BaseHyperParameters):
    exposure_set_size: int
    weight_decay: float

class OE_Experiment(Experiment):

    hp: HyperParams
    network: MySimpleMLP

    classifier_weight: float = 50.0

    def __init__(self, hp: HyperParams) -> None:
        super().__init__(hp)
        self.classes_seen: typing.Set[int] = set()
        self.after_eval_forward = self.after_forward

    def make_network(self) -> nn.Module:
        return MySimpleMLP()

    def make_optimizer(self, parameters) -> torch.optim.Optimizer:
        optimizer = torch.optim.SGD(parameters, self.hp.lr, weight_decay=self.hp.weight_decay)
        return optimizer

    def make_criterion(self):
        return softTargetCrossEntropy

    def make_scenario(self):
        # transform = transforms.Compose(
        #     [transforms.ToTensor(), transforms.Normalize((0.2860,), (0.3530,)), transforms.Resize(32)]
        # )
        scenario: NCScenario = SplitMNIST(
            n_experiences=5,
            fixed_class_order=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            dataset_root=DATASETS)


        self.outliers = AvalancheDataset(
            torchvision.datasets.KMNIST(DATASETS, train=True, download=True, 
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1918,), (0.3483,))
            ]))
        )

        # self.outliers += AvalancheDataset(
        #     torchvision.datasets.MNIST(DATASETS, train=True, download=True, 
        #     transform=transforms.Compose([
        #         transforms.ToTensor(),
        #         transforms.Normalize((0.1307,), (0.3081,)),
        #         transforms.RandomRotation((90, 360-90)),
        #     ]))
        # )


        # Use a uniform distribution for outlier exposure
        return scenario

    def add_plugins(self):
        return [LwFPlugin()]
        

    def known_known(self, n_classes: int):
        def _known_known(y: int) -> Tensor:
            new_y = torch.zeros(n_classes)
            new_y[y] = 1.0
            return new_y
        return _known_known

    def known_unkown(self, n_classes: int, classes_seen: typing.Set[int]):
        def known_unkown(y: int) -> Tensor:
            # return y
            y = torch.ones(n_classes) * 1/(n_classes-len(classes_seen))

            for seen in classes_seen:
                y[seen] = 0.0

            return y
        return known_unkown

    def after_train_dataset_adaptation(self, strategy: SupervisedTemplate):
        scenario: NCScenario = strategy.experience.benchmark
        current_dataset: AvalancheSubset = strategy.adapted_dataset

        n_classes: int = scenario.n_classes
        self.classes_seen.union(strategy.experience.classes_in_this_experience)
        # Build set of outlier exposures
        exposure_set = self.outliers.add_transforms(
            target_transform=self.known_unkown(n_classes, self.classes_seen))

        current_dataset = current_dataset.add_transforms(target_transform=self.known_known(n_classes))

        # Random indices ordering 
        indices = list(range(len(exposure_set)))
        shuffle(indices)
        indices = indices[:self.hp.exposure_set_size]
        exposure_set = AvalancheSubset(exposure_set, indices)
        strategy.adapted_dataset = exposure_set + current_dataset

    def after_eval_dataset_adaptation(self, strategy):
        n_classes =  strategy.experience.benchmark.n_classes
        strategy.adapted_dataset = strategy.adapted_dataset.add_transforms(
            target_transform=self.known_known(n_classes))




In [94]:

experiment = OE_Experiment(
    HyperParams(
        lr=0.01,
        train_mb_size=64,
        train_epochs=50,
        eval_mb_size=128,
        eval_every=-1,
        device="cuda",
        exposure_set_size=12000,
        weight_decay=0.0
    )
).train()

/Scratch/al183/avalanche/avalanche/training/templates/base.py:200: UserWarning: Plugin <__main__.OE_Experiment object at 0x7fbda8272430> implements incompatible callbacks for template <avalanche.training.templates.supervised.SupervisedTemplate object at 0x7fbd970dccd0>. This may result in errors.
  warnings.warn(


Start of experience: 0
Current Classes:     [0, 1]
Experience size:     12665
Start of experience: 1
Current Classes:     [2, 3]
Experience size:     12089
Start of experience: 2
Current Classes:     [4, 5]
Experience size:     11263
Start of experience: 3
Current Classes:     [6, 7]
Experience size:     12183
Start of experience: 4
Current Classes:     [8, 9]
Experience size:     11800
